#### Resampling Methods
##### Involves repeatedly drawing samples from a training set and refitting a model of interest on each sample in order to obtain additional info about the fitted model.
##### Two most common resampling methods: ***Cross-validation*** and the ***Bootstrap***
##### Cross-validation can be used to estimate the ***test error*** associated with a given statistical learning method in order to evaluate its performance, or to select the appropriate level of flexibility.
##### The process of evaluating a model's performance is known as ***model assessment***
##### ***The Validation Set Approach:***
##### --Involves randomly dividing the available set of observations ito two parts : ***training set***, and ***validation set or hold-out set***
##### The model is fit on the training set, and the fitted model is used to predict the responses for the observations in the validation set. The resulting validation set error rate is typically assessed using ***MSE** in the case of a quantitative response.
##### ***Leave-One-Out Cross-Validation*** 
##### ***k-Fold Cross-Validation*** --randomly dividing the set observations into k groups or folds, of approximately equal size. The first fold is treated as a validation set, and the method is fit on the remaining (k-1) folds.
##### ***The Bootstrap*** --can be used to quantify the uncertainity associated with a given estimator or statistical learning method.
##### The bootstrap can be used to estimate the ***standard errors*** of the coefficients from a linear regression fit.


In [199]:
!pip install numpy pandas matplotlib statsmodels

In [200]:
!pip install ISLP

In [201]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

#### The Validation Set Approach
##### We explore the use of the validation set approach in order to estimate the test error rates that result from fitting various linear models on the Auto data set.
##### we have 396 observations , we split into two equal sets of size 196 
##### set a random seed so that results obtained can be reproduced precisely ata later time.

In [202]:
Auto= load_data("Auto")
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [203]:
#Now we fit a linear regression using only the observations corresponding to the training set Auto_train
hp_mm= MS(["horsepower"])
X_train= hp_mm.fit_transform(Auto_train)
y_train= Auto_train["mpg"]
model = sm.OLS(y_train, X_train)
results= model.fit()

In [204]:
#We now use the predict() method of results evaluated on the model matrix for this model created using teh validation data set.
X_valid= hp_mm.transform(Auto_valid)
y_valid= Auto_valid["mpg"]
valid_pred= results.predict(X_valid)
np.mean((y_valid - valid_pred)**2) #MSE of our model

23.616617069669882

In [205]:
#WE can also estimate the validation error for higher-degree ploynomial regressions
# we first provide a function evalMSE() that takes a model string as well as training and test set and returns the MSE on the test set
def evalMSE(terms, response, train, test):
    mm= MS(terms)
    X_train= mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]
    results= sm.OLS(y_train, X_train).fit()
    test_pred= results.predict(X_test)
    return np.mean((y_test - test_pred)**2) #MSE of our model

In [206]:
#lets use this function to estimate the validation MSE using linear, quadratic and cubic fits.
#we use enumerate() which gives both the values and indices of objects as one iterates over a loop
MSE = np.zeros(3) #creates an array to store results
for idx, degree in enumerate(range(1,4)): #the loop iterates over polynomial degrees 1-3 and updates MSE
    MSE[idx] = evalMSE([poly("horsepower", degree)], "mpg", Auto_train, Auto_valid)
MSE    

array([23.61661707, 18.76303135, 18.79694163])

In [207]:
#If we choose different training/validation split instead, we can expect different errors on the validation set
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly("horsepower", degree)], "mpg", Auto_train, Auto_valid)
MSE  

###  A model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is no evidence of an improvement in using a cubic function of horsepower.

array([20.75540796, 16.94510676, 16.97437833])

#### Cross-Validation

##### What is a wrapper?
##### Its like a translator or adapter that lets two incompatible tools work together
##### Example For task A : you fit a model using statsmodels(e.g logistic regression) and task B you want to use scikit-learn's cross-validation tools.
##### Problem: statsmodels and scikit-learn don't natively understand each other's formats
##### Therefore sklearn_sm() wrapper acts as the adapter
##### ***Key components of sklearn_sm()***
##### Input: a ***statsmodels*** model(e.g, logit)
##### Optional Arguments: ***model_str***: formula(e.g "mpg~ horsepower + weight").
#####                     ***model_args***: e.g {"family":sm.families.Binomial() for       logistic regression}

In [208]:
#Just as an illustration
#from ISLP import sklearn_sm
#import statsmodels.api as sm
#from sklearn.model_selection import cross_val_score

#step1: Define the wrapper
#logit_wrapper= sklearn_sm(
 #   sm.Logit,  #statsmodels model
  #  model_str= "mpg01~ horsepower + weight", #formula
   # model_args={"family": sm.families.Binomial()} #Logistic regression
#)

#step2: Use with scikit_learn's cross_validation
#scores = cross_val_score(
 #   logit_wrapper,  #wrapped model
  #  Auto,  #Data
   # Auto["mpg01"], #target
    #cv=5    #5-fold cross_validation
#)

##### Here is our wrapper in function

In [209]:
hp_model = sklearn_sm(sm.OLS, MS(["horsepower"]))
X, Y = Auto.drop(columns=["mpg"]), Auto["mpg"]
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results["test_score"])
cv_err

# hp_model : an object with appropriate fit(), predict() and score()
# cv : specifies an integer K results in K-fold cross-validation
# cv=Auto.shape[0] : provides value corresponding to the total number of observations, 
# which results in leave-one-out cross-validation(LOOCV)

24.231513517929233

##### We can repeat the procedure for increasingly complex polynomial fit
##### To automate the process, we again use a for loop which iteratively fits polynomial regressions of degree 1 to 5, computes the associated cross-validation error, and stores it in the ith element of the vector cv_error.

In [210]:
cv_error = np.zeros(5)
H= np.array(Auto["horsepower"]) #array of horsepower values
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d + 1)) #creates polynomial features up to degree d for the horse values: computes element_wise power operations to create the design matrix
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0]) #LOOCV
    cv_error[i] = np.mean(M_CV["test_score"]) #average MSE
cv_error    

## We see a sharp drop in the estimated test MSE between linear models and quadratic fits, but then no clear improvement using higher-degree polynomials

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.0332262 ])

##### In the CV example above we used K = n , we can also use K < n. Here we use KFold() to partition the data into K = 10 random groups.

In [211]:
cv_error = np.zeros(5)
cv= KFold(n_splits=10, shuffle=True, random_state=0)
for i, d in enumerate(range(1,6)):
    X= np.power.outer(H, np.arange(d + 1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV["test_score"])
cv_error    

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719075])

##### The cross-validate() function is flexible and can take different splitting mechanisms as an argument.
##### For instance, one can use ShuffleSplit() function to implement the validation set approach just as easily as K-fold cross-validation

In [212]:
validation = ShuffleSplit(n_splits=1,
                         test_size=196,
                         random_state=0)
results= cross_validate(hp_model,
                       Auto.drop(["mpg"], axis=1),
                       Auto["mpg"],
                       cv= validation)
results["test_score"]

array([23.61661707])

##### Lets estimate the variability in the test error.

In [213]:
validation = ShuffleSplit(n_splits=10,
                         test_size=196,
                         random_state=0) #randomly splits data multiple times for robust validation
results = cross_validate(hp_model,
                        Auto.drop(["mpg"], axis=1),
                        Auto["mpg"],
                        cv=validation)
results["test_score"].mean(), results["test_score"].std()

#Note that this SD is not a valid estimate of the sampling variability of the mean test score or the individual scores, since the randomly-selected training samples overlap and hence introduce correlations.
# But it does give an idea of the Monte Carlo variation incurred by picking different random folds.

(23.802232661034164, 1.4218450941091862)

### The Bootstrap

##### Estimating the Accuracy of a Statistic of Interest

##### We use the Portfolio data set in the ISLP package .The goal is to estimate the sampling variance of the parameter (coefficients).
##### We will create a function alpha_func(), which takes as input a dataframe D assumed to have columns X and Y, as well as vector idx indicating which observations should be used to estimate the weight(coefficient). The function then outputs the estimate for (coefficients) based on the selected observations.

In [214]:
Portfolio = load_data("Portfolio")
def alpha_func(D, idx):
    cov_ = np.cov(D[["X", "Y"]].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1])/(cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))
# assume the Portfolio has two columns: 
# X: returns of asset X
# Y: returns of asset Y
# the alpha_func computes an optimal portfolio weight(α) that miinimizes variance
#The function calculates the cov_: covariance of matrix of X and Y for the resampled data (idx)
# The formula gives the weight (α) to invest in X for minimum portfolio risk: α = [Var(Y) - Cov(X,Y)] / [Var(X) + Var(Y) - 2*Cov(X,Y)]

            # Set up bootstrap
#n_bootstraps = 1000
#alpha_estimates = np.zeros(n_bootstraps)

#for i in range(n_bootstraps):
    # Resample indices with replacement
 #   idx = np.random.choice(len(Portfolio), len(Portfolio), replace=True)
    # Compute alpha for this resample
  #  alpha_estimates[i] = alpha_func(Portfolio, idx)

# Results
#print(f"Mean α: {np.mean(alpha_estimates):.3f}")
#print(f"95% CI: [{np.percentile(alpha_estimates, 2.5):.3f}, {np.percentile(alpha_estimates, 97.5):.3f}]")
#print(f"95% CI: [{np.percentile(alpha_estimates, 2.5):.3f}, {np.percentile(alpha_estimates, 97.5):.3f}]")

In [215]:
# lets estimate (α) using all 100 observations
alpha_func(Portfolio, range(100))

0.57583207459283

In [216]:
#NExt we randomly select 100 observations from range(100), with replacement.
#This is equivalent to constructing a new bootstrap data set and recomputing alpha based on new data
rng = np.random.default_rng(0) #creates random  number generator object with a fixed seed(0) for reproducibility
alpha_func(Portfolio, 
          rng.choice(100,
                    100,
                    replace=True))

0.6074452469619004

In [217]:
#This process can be generalized to create a simple function boot_SE() for computing the bootstrap standard error for arbitrary functions that take only a data frame as an argument  

def boot_SE(func,
          D,
          n=None,
          B=1000,
          seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n if n is not None else D.shape[0] #/ n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                        n,
                        replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B) **2)    

In [218]:
#Lets use our function to evaluate the accuracy of our estimate of alpha using B = 1,000 bootstrap replications
alpha_SE= boot_SE(alpha_func,
                 Portfolio,
                 B= 1000,
                 seed=0)
alpha_SE

0.09118176521277699

#### Estimating the Accuracy of a Linear Regression Model

##### The bootstrap approach can be used to assess the variability of the coefficient estimates and predictions from a statistical learning method. 
##### Here we use the bootstrap approach to assess the variability of the estimates for B0 and B1,the intercept and slope terms for the linear regression model that uses horsepower to predict mpg in the Auto data set

In [219]:
#Fit an OLS regression on a bootstrap sample of the data
def boot_OLS(model_matrix, response, D, idx):
    D_= D.loc[idx]
    Y_= D_[response]
    X_= clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [220]:
#partially applied version of boot_OLS : fixes model_matrix=MS(["horsepower"]) and response= "mpg"
#Now only requires D and idx to run
hp_func = partial(boot_OLS, MS(["horsepower"]), "mpg")

In [221]:
#the hp_func() can now be used in order to create bootstrap estimates for the intercept and slope terms by randomly sampling from among the observations with replacement.
#We first demo its utility on 10 bootstrap samples
rng = np.random.default_rng(0)
#get the actual indices from the Auto dataframe
indices = Auto.index.tolist()
np.array([hp_func(Auto,
                 rng.choice(indices, len(indices), replace=True)) for _ in range(10)])

array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

In [222]:
#We use boot_SE() to compute the standard errors of 1000 bootstrap estimates for the intercept and slope terms
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

In [223]:
#standard formulas can be used to compute the standard errors for the regression coefficients in a linear model
hp_model.fit(Auto, Auto["mpg"])
model_se = summarize(hp_model.results_)["std err"]
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [224]:
#we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model to the data
quad_model = MS([poly("horsepower", 2, raw=True)])
quad_func= partial(boot_OLS, quad_model, "mpg")
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [225]:
#we compare the results to the SE computed using sm.OLS()
M= sm.OLS(Auto["mpg"], quad_model.fit_transform(Auto))
summarize(M.fit())["std err"]

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64

#### Question 1
##### In Classification chapter we used logistic regression to predict the probability of default using ***income*** and ***balance*** on ***Default*** data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning the analysis.

##### a). Fit a logistic regression model that uses income and balance to predict default

In [226]:
Default = load_data("Default")
dropVar=Default.columns.drop(["default", "student"])
y= Default.default == "Yes"
X = MS(dropVar).fit_transform(Default)
model= sm.GLM(y,X, family = sm.families.Binomial())
results= model.fit()
summarize(results)


,coef,std err,z,P>|z|
intercept,-11.540500,0.435000,-26.544,0.0
balance,0.005600,0.000000,24.835,0.0
income,0.000021,0.000005,4.174,0.0


##### b). Using the validation set approach, estimate the test error of this model. Inorder to do this, you must perform the following steps:

##### ***i***. Split the sample set into a training set and validation set.

In [227]:
default_train, default_valid = train_test_split(Default, test_size=5000, random_state=0)

##### ***ii*** Fit a multiple regression model using only the training observations

In [228]:
var=MS(dropVar)
X_train = var.fit_transform(default_train)
y_train= default_train["default"]=="Yes"
model= sm.GLM(y_train, X_train, family= sm.families.Binomial())
results= model.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-11.389600,0.635000,-17.935,0.000
balance,0.005600,0.000000,16.792,0.000
income,0.000016,0.000007,2.151,0.031


##### ***iii*** Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5

In [229]:
#preprocess the validation data using the same transformer var
X_valid = var.transform(default_valid)
y_valid = default_valid["default"]== "Yes"
valid_pred = results.predict(X_valid)
valid_pred_class = (valid_pred>0.5)
valid_pred_class

9394    False
898     False
2398    False
5906    False
2343    False
        ...  
3996    False
5889    False
4577    False
8600    False
847     False
Length: 5000, dtype: bool

##### ***iv*** Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified

In [230]:
from ISLP import confusion_table
confusion_table(y_valid, valid_pred_class)

Truth,False,True
Predicted,,
False,4801,13
True,132,54


In [231]:
#the set error
np.mean(y_valid!=valid_pred_class)

0.029

##### c) Repeat the process in (b) three times, using three diffrent splits of the observations into a training set and a validation set. comment on the results obtained

In [232]:
#store errors for each split
set_error= np.zeros(3)

for i in range(3): # repeat 3 times with different splits
    default_train, default_valid = train_test_split(Default, test_size=5000, random_state=i)
    #preprocess data
    X_train= var.fit_transform(default_train)
    y_train= default_train["default"] == "Yes"
    X_valid = var.transform(default_valid)
    y_valid= default_valid["default"] == "Yes"
    #train model
    model= sm.GLM(y_train, X_train, family= sm.families.Binomial())
    results= model.fit()
    #predict on validation set
    valid_pred= results.predict(X_valid)
    valid_pred_class= (valid_pred>0.5)
    #compute validation error
    set_error[i]= np.mean(y_valid!=valid_pred_class)
set_error    

#Comments: The errors are very close indicating that the model's performance is stable across different training-validation splits
#No major overfitting - since errors do not spike in any split

array([0.029 , 0.025 , 0.0248])

##### d). Now consider a logistic regression model that predicts the probabilty of default using income, balance and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leand to a reduction in the test error rate

In [233]:
dropVar = Default.columns.drop(["default"])
design= MS(dropVar)
X_train= design.fit_transform(default_train)
y_train= default_train["default"] == "Yes"
X_valid= design.transform(default_valid)
y_valid= default_valid["default"]== "Yes"
model = sm.GLM(y_train, X_train, family= sm.families.Binomial())
results= model.fit()
#predict on validation set
valid_pred= results.predict(X_valid)
valid_pred_class = (valid_pred>0.5)
np.mean(y_valid!=valid_pred_class)

#Adding the student dummy variable slightly reduces the test error
#The improvement is marginal suggesting that student status has a weak but non-zero effect on default prediction


0.0254

#### Question 2
##### We continue to consider the use of a logistic regression model to predict the probability of ***default*** using ***income*** and ***balance*** on the ***Default*** data set. In particular, we will now compute estimates for the standard errors of the ***income*** amd ***balance*** logistic regression coefficients in two different ways:
##### 1) using the bootstrap and
##### 2) using the standard formula for computing the standard errors in the GLM(). Do not forget to set a random seed before beginning your analysis

##### ***(a)*** Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors

In [234]:
dropVar= Default.columns.drop(["default", "student"])
X= MS(dropVar).fit_transform(Default[dropVar])
y= Default["default"] == "Yes"
model= sm.GLM(y, X, family=sm.families.Binomial())
results= model.fit()
summarize(results)["std err"]

intercept    0.435000
balance      0.000000
income       0.000005
Name: std err, dtype: float64

##### ***(b)*** Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple regression model

In [235]:
def boot_fn(predictors, response, D, idx):
    D_ = D.loc[idx]
    Y_ =D_[response]
    X_ = clone(predictors).fit_transform(D_)
    model= sm.GLM(Y_, X_, family= sm.families.Binomial()).fit()
    return model.params

##### ***(c)*** Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefficients for income and balance

In [236]:
#convert default from strings to binary
Default["default"] = Default["default"].map({"No":0, "Yes":1})
bal_inc_func= partial(boot_fn, MS(["balance", "income"]), "default")
rng= np.random.default_rng(0)
#get actual indices
indices= Default.index.tolist()
np.array([bal_inc_func(Default, rng.choice(indices, len(indices), replace=True))])

array([[-1.16416373e+01,  5.73877605e-03,  1.87775777e-05]])

In [245]:
#generate multiple bootstrap samples and collect results
B= 1000
#initialize boot_coefs as a 2D array to store multiple coeffs assuming there aew 3 coefficients (intercept, balance, income)
boot_coefs=np.zeros((B, 3))
for i in range(B):
    #get bootstrap sample indices
    boot_idx = rng.choice(indices, len(indices), replace=True)
    #get coeffs
    results = bal_inc_func(Default, boot_idx)
    #convert to numpy array and store
    coeff_array= np.array(results)
    boot_coefs[i, :]=coeff_array
#calculate standard errors from bootstrap samples
SE = boot_coefs.std(axis=0)
print("Intercept SE:", SE[0] )
print("Balance SE:", SE[1])
print("Income SE:", SE[2])

#The bootstrap method yields lower SE compared to sm.GLM()
    

Intercept SE: 0.4552564131999408
Balance SE: 0.00023821330741163237
Income SE: 5.0309299677176e-06


#### Question 3
##### We say that ***cross_validate()*** function  can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just sm.GLM() and the predict() method of the fitted model within a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the ***Weekly*** data set. 

##### ***(a)*** Fit a logistic regression model that predicts ***Direction*** using ***Lag1*** and ***Lag2***.

In [277]:
Weekly = load_data("Weekly")
#Weekly.Direction.values
dropVar= Weekly.columns.drop(["Year", "Lag3", "Lag4", "Lag5", "Volume", "Today", "Direction"])
Weekly["Direction_binary"] = Weekly["Direction"].map({"Down":0, "Up":1})
X= MS(dropVar).fit_transform(Weekly)
y= Weekly["Direction_binary"]
model= sm.GLM(y, X, family= sm.families.Binomial())
results= model.fit()
summarize(results)



,coef,std err,z,P>|z|
intercept,0.2212,0.061,3.599,0.000
Lag1,-0.0387,0.026,-1.477,0.140
Lag2,0.0602,0.027,2.270,0.023


##### ***(b)*** Fit a logistic regression model that predicts Direction using Lag1 and Lag2 using all but the first observation

In [278]:
#Get all observations except the first one
X_loocv= X.iloc[1:]
y_loocv = y.iloc[1:]
#fit the model on all but first observation
model_loocv= sm.GLM(y_loocv, X_loocv, family= sm.families.Binomial())
results_loocv= model_loocv.fit()
summarize(results_loocv)

,coef,std err,z,P>|z|
intercept,0.2232,0.061,3.630,0.000
Lag1,-0.0384,0.026,-1.466,0.143
Lag2,0.0608,0.027,2.291,0.022


##### ***(c)*** Use the model from (b) to predict the direction of the first observation. You can do this by predicting that the first observation will go up if ***P(Direction= "Up"|Lag1,Lag2)>0.5***. Was this observation correctly classified?

In [302]:
#predict the left-out observation
first_obs= X.iloc[[0]]
pred_prob= results_loocv.predict(first_obs)
pred_class= 1 if pred_prob[0] > 0.5 else 0
#compare with actual class
actual_class = y.iloc[0]
correct = pred_class==actual_class
correct



False

##### ***(d)*** Write a for loop from i=1 to i=n, where n is the number of observations in the data set, that performs each of the following steps:

##### ***i.*** Fit a logistic regression model using all but the ***ith*** observation to predict Direction using Lag1 and Lag2
##### ***ii.*** Compute the posterior probability of the market moving up for the ***ith*** observation.
##### ***iii*** Use the posterior probability for the ***ith*** observation in order to predict whether or not the market moves up.
##### ***iv*** Determine whether or not an error was made in predicting the direction for the ***ith*** observation. If an error was made, then indicate this as 1, and otherwise indicate it as 0.

##### ***(e)*** Take the average of the n numbers obtained in (d)iv in order to obtain the LOOCV estimate for the test error. Comment on the results.

In [308]:
#initialize error counter
errors=0
n= len(Weekly)
for i in range(1, n):
    #create training data(all but ith observation)
    X_train= X.drop(i)
    y_train= y.drop(i)
    #fit log regression
    model= sm.GLM(y_train, X_train, family= sm.families.Binomial())
    result = model.fit()  # Need to fit the model first

    #predict left_out observation
    X_test= X.iloc[[i]]
    pred_prob= result.predict(X_test)  # Use the fitted model result to predict
    pred_class= 1 if pred_prob.iloc[0] > 0.5 else 0

    #compare with actual
    actual = y.iloc[i]
    if pred_class != actual:
        errors +=1
#Calculate LOOCV error rate     
loocv_error= errors / n
print(f"LOOCV error rate: {loocv_error:.4f} ({errors} errors out of {n} predictions)")

LOOCV error rate: 0.4490 (489 errors out of 1089 predictions)
